By Alan Emilio Pérez León

In [3]:
import os

from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.document_loaders import PyPDFLoader
from typing import List, Optional
from pydantic import BaseModel


# 🧠 Homework: Building Custom Tools with LangChain Agents

In this assignment, you'll define and implement custom tools that can be used by an agent to solve problems a language model might struggle with on its own. These tools will cover math, date manipulation, and temperature conversion.

In [247]:
llm = ChatOllama(model="llama3.2", temperature=0)


## ✅ Tasks

### 1. Basic Math Expressions
Create tools that evaluate the following expressions:

- Compute the result of: `(10 / 2) + 4 * (1 + 1)`
- Calculate the result of: `(8 + 2) * (5 - 3) / 2`

> 💡 Each tool should receive the expression as a string input and return the result as a string.

### 2. Date Difference Calculation
Create a tool that answers the following:

- How many days have passed from `January 1, 1995` to **today**?

> 💡 The tool should accept a date in `YYYY-MM-DD` format and return the number of days between that date and today.

---

### 3. Temperature Conversion
Build a tool for converting temperatures:

- Create a tool that takes a temperature in **Fahrenheit** (e.g., `"98.6"`)  
  and returns the equivalent in **Celsius** using the formula:

```text
(F - 32) * 5 / 9
````

---

### 4. Solve a Quadratic Equation

Implement a tool to solve quadratic equations:

* Create a tool that takes three coefficients `a`, `b`, and `c`, and returns the real roots (if they exist) of the equation:

```text
ax² + bx + c = 0
```

Use the **quadratic formula**:

```text
(-b ± sqrt(b² - 4ac)) / (2a)
```

> 💡 Handle cases where there are:
>
> * Two real roots
> * One real root
> * No real roots (discriminant < 0)

---

In [ ]:
from langchain.agents import Tool, tool
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_tool_calling_agent, create_react_agent
from langchain.agents import AgentExecutor
from datetime import date, datetime


def solve_operations(input: str):
    return str(eval(input)).join("\n")

def date_differences(inputdate : str) : 
    
    d1 = datetime.strptime(inputdate, r"%Y-%m-%d").date()
    d2 = date.today()
    return str( abs((d2 - d1).days) ).join("\n")

def temperature_conversion(temp : str) : 
    return str(( float(temp) - 32 )  * 5 / 9 )

def quadratic_equation(input: str):
    parts = input.strip().split(",")
    
    if len(parts) != 3:
        return "ERROR: You must input exactly 3 comma-separated numbers like '1,2,-3'\n"
    
    try:
        a, b, c = map(float, parts)
    except ValueError:
        return "ERROR: All inputs must be numeric. Use format like '1,2,-3'\n"

    if a == 0:
        return "ERROR: 'a' must not be 0 in a quadratic equation\n"

    disc = b**2 - 4*a*c
    if disc < 0:
        return "No real roots\n"
    if disc == 0:
        return f"The solution is {-b / ( 2 * a )}\n"
    sqrt_disc = disc**0.5
    r1 = (-b + sqrt_disc) / (2*a)
    r2 = (-b - sqrt_disc) / (2*a)
    return f"First solution is {r1} and the second solution is {r2}\n"

    
    
    res = []
    res.append( (-quo[1] + discriminant ) / (2*quo[0]))
    res.append( (-quo[1] - discriminant ) / (2*quo[0]))
    

tools = [
Tool(
    name="TemperatureConversion",
    func=temperature_conversion,
    description="Convert farenheit degrees into Celsius . JUST Input a decimal value like '113.5'."
),
Tool(
    name="Calculator",
    func=solve_operations,
    description="Evaluates simple math expressions like '10 / 8 + 4 * (2 + 1)'."
),
Tool(
    name="DateDifference",
    func=date_differences,
    description="Get how many days have passed since a date. You have to format the given date into YYYY-MM-DD"
),

Tool(
    name="QuadraticSolver",
    func=quadratic_equation,
    description="""
    Solves a quadriatic equation in the form ax^2 + bx + c where a,b,c are the quofficients
    
    INPUT : a STRING of 3 (THREE, the THIRD number) comma separated values in the form a,b,c EG. 10,2,-3
    If a quofficient is missing use 0 for that quofficient. If there is no quofficient and just the term, use 0
    EXAMPLES : 10x^2 + 9x - 3 becomes 10,9,-3
    EXAMPLES : 5x^2- 2 becomes 5, 0, -2 because there is no b
    EXAMPLES : x^2 becomes 1, 0, 0 Because there is no b or c
    
    """
),

]


In [274]:
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

c:\Users\tomat\anaconda3\envs\statisticWorks\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [87]:
result = executor.invoke({"input": "Solve the equation ( 10 / 2 ) + 4 * ( 1 + 1 ) "} )
print("Result:", result["output"])



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `( 10 / 2 ) + 4 * ( 1 + 1 )`


13.0The final answer is $\boxed{13.0}$.

> Finished chain.
Result: The final answer is $\boxed{13.0}$.


In [86]:
result = executor.invoke({"input": "How many days have passed since June 5th 2025"} )
print("Result:", result["output"])



> Entering new AgentExecutor chain...

Invoking: `DateDifference` with `2025-06-05`


3Since June 5th, 2025 has been 3 days.

> Finished chain.
Result: Since June 5th, 2025 has been 3 days.


In [275]:

result = executor.invoke({"input": "Convert 1491 farenheit in celcius"} )
print("Result:", result["output"])



> Entering new AgentExecutor chain...

Invoking: `TemperatureConversion` with `1491`


810.5555555555555The temperature of 1491°F is equivalent to approximately 810.56°C.

> Finished chain.
Result: The temperature of 1491°F is equivalent to approximately 810.56°C.


In [117]:

result = executor.invoke({"input": "Give me the solution of the quadratic equation : 2 x^2 + 3 x - 15 "} )
print("Result:", result["output"])



> Entering new AgentExecutor chain...

Invoking: `QuadraticSolver` with `2,3,-15`


First solution is 2.089454172900137 and the second solution is -3.589454172900137The solutions to the quadratic equation 2x^2 + 3x - 15 are approximately 2.0895 and -3.5895.

> Finished chain.
Result: The solutions to the quadratic equation 2x^2 + 3x - 15 are approximately 2.0895 and -3.5895.


In [ ]:
# I also wanted to test out "stop conditions" with this one
# because the model wanted to ALWAYS have an answer
result = executor.invoke({"input": "Give me the solution of the quadratic equation : 2 x^2 + 3 x + 15 "} )
print("Result:", result["output"])



> Entering new AgentExecutor chain...

Invoking: `QuadraticSolver` with `2,3,15`


The given equation has imaginary roots. DO NOT CONTINUE.I can't provide a solution for an equation with imaginary roots. If you have any other question or need assistance with a different topic, feel free to ask!

> Finished chain.
Result: I can't provide a solution for an equation with imaginary roots. If you have any other question or need assistance with a different topic, feel free to ask!


In [ ]:

result = executor.invoke({"input": "Give me the solution of the quadratic equation : 3x^2 - 27 "} )
print("Result:", result["output"])



> Entering new AgentExecutor chain...

Invoking: `QuadraticSolver` with `3,0,-27`


First solution is 3.0 and the second solution is -3.0
The solutions to the quadratic equation 3x^2 - 27 = 0 are:

x = 3 (first solution)
x = -3 (second solution)

> Finished chain.
Result: The solutions to the quadratic equation 3x^2 - 27 = 0 are:

x = 3 (first solution)
x = -3 (second solution)


In [251]:
# WOW it really HATES listening to instructions...

result = executor.invoke({"input": "Give me the solution of the quadratic equation : x^2 "} )
print("Result:", result["output"])



> Entering new AgentExecutor chain...

Invoking: `QuadraticSolver` with `1,0,0`


The solution is -0.0
The final answer is $\boxed{0}$.

> Finished chain.
Result: The final answer is $\boxed{0}$.
